In [ ]:
import os
import numpy as np
import skimage.io as io
from pycocotools import coco, mask as maskUtils
from PIL import Image
import torch
from torchvision import transforms
import cv2

# Load annotation files
rgbAnnFile = './aauRainSnow-rgb.json'
thermalAnnFile = './aauRainSnow-thermal.json'
rainSnowRgbGt = coco.COCO(rgbAnnFile)
rainSnowThermalGt = coco.COCO(thermalAnnFile)

def coco_ann_to_mask(ann, height, width):
    seg = ann.get('segmentation', None)
    if not seg:
        raise ValueError("Missing segmentation")
    if isinstance(seg, list):
        if len(seg) == 0:
            raise ValueError("Empty segmentation list")
        rles = maskUtils.frPyObjects(seg, height, width)
        rle = maskUtils.merge(rles)
    elif isinstance(seg, dict) and 'counts' in seg:
        rle = seg
    else:
        raise ValueError("Invalid segmentation format")
    return maskUtils.decode(rle)

def build_aau_dataset(rainSnowRgbGt, rainSnowThermalGt, target_size=(224, 224), max_images=2197):
    dataset = []

    tf_rgb = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize(target_size),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5]*3, std=[0.5]*3)
    ])

    tf_thermal = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize(target_size),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5], std=[0.5])
    ])

    for i in range(max_images):
        annIds = rainSnowRgbGt.getAnnIds(imgIds=[i])
        anns = rainSnowRgbGt.loadAnns(annIds)

        if not anns:
            continue

        rgb_info = rainSnowRgbGt.loadImgs([i])[0]
        thermal_info = rainSnowThermalGt.loadImgs([i])[0]
        thermal_anns = rainSnowThermalGt.loadAnns(annIds)

        try:
            rgb_img = io.imread('./' + rgb_info['file_name'])
            thermal_img = io.imread('./' + thermal_info['file_name'])
        except Exception as e:
            print(f"Error reading image {i}: {e}")
            continue

        height, width = rgb_img.shape[:2]
        combined_mask = np.zeros((height, width), dtype=np.uint8)

        for ann in anns:
            try:
                cat_id = ann['category_id']
                mask = coco_ann_to_mask(ann, height, width)
                combined_mask = np.maximum(combined_mask, mask * cat_id)
            except Exception as e:
                print(f"Skipping ann {ann.get('id')} in image {i}: {e}")

        if len(thermal_img.shape) == 3:
            thermal_img = cv2.cvtColor(thermal_img, cv2.COLOR_BGR2GRAY)

        try:
            rgb_tensor = tf_rgb(rgb_img)
            thermal_tensor = tf_thermal(thermal_img)
            mask_tensor = torch.from_numpy(np.array(
                transforms.Resize(target_size, interpolation=Image.NEAREST)(
                    Image.fromarray(combined_mask)
                )
            )).long()
        except Exception as e:
            print(f"Error transforming image {i}: {e}")
            continue

        dataset.append({
            'id': i,
            'file_name': rgb_info['file_name'],
            'rgb': rgb_tensor,
            'thermal': thermal_tensor,
            'mask': mask_tensor
        })

    print(f"Dataset built with {len(dataset)} samples.")
    return dataset


In [ ]:
dataset = build_aau_dataset(rainSnowRgbGt, rainSnowThermalGt, target_size=(224, 224))

# Print the first few samples
for sample in dataset[:3]:
    print(f"\nID: {sample['id']}, File: {sample['file_name']}")
    print(f"RGB: {sample['rgb'].shape}, Thermal: {sample['thermal'].shape}, Mask: {sample['mask'].shape}")


loading annotations into memory...
Done (t=0.09s)
creating index...
index created!
loading annotations into memory...
Done (t=0.27s)
creating index...
index created!
Error reading image Egensevej/Egensevej-1/cam1-00055.png or Egensevej/Egensevej-1/cam2-00055.png: No such file: 'c:\Users\Tricks\Documents\Shared Repository\dual-vit-vehicle\Egensevej\Egensevej-1\cam1-00055.png'
Error reading image Egensevej/Egensevej-1/cam1-00141.png or Egensevej/Egensevej-1/cam2-00141.png: No such file: 'c:\Users\Tricks\Documents\Shared Repository\dual-vit-vehicle\Egensevej\Egensevej-1\cam1-00141.png'
Error reading image Egensevej/Egensevej-1/cam1-00218.png or Egensevej/Egensevej-1/cam2-00218.png: No such file: 'c:\Users\Tricks\Documents\Shared Repository\dual-vit-vehicle\Egensevej\Egensevej-1\cam1-00218.png'
Error reading image Egensevej/Egensevej-1/cam1-00219.png or Egensevej/Egensevej-1/cam2-00219.png: No such file: 'c:\Users\Tricks\Documents\Shared Repository\dual-vit-vehicle\Egensevej\Egensevej-1\ca